In [1]:
from openai import OpenAI
import json, os, sys
import os
import re
import random

In [2]:
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [3]:
def generate_difference(chunk, client, model="gpt-4o"):
    prompt = f"""
You are an expert in news analysis and skilled at capturing narrative difference. 
I have some news articles in English and Chinese and I need you to help me identify the narrative differences between them.

{chunk}


"""
    messages = [{"role": "user", "content": prompt}]
    try:
        response = client.chat.completions.create(
            model=model, messages=messages, temperature=0
        )

        content = response.choices[0].message.content
        return content
    except Exception as e:  # if the model fails to return a response
        print(f"Error: {e}")
        return "Sorry, error from GPT."

In [4]:
def process_batches(input_file, output_file, client, model, label_a_index):
    # Read the article from the input JSON file
    with open(input_file, "r") as f:
        articles = json.load(f)

    # Extract summaries from the articles
    article_body = [article["body"] for article in articles]

    # get the article body with the label_a_index list

    article_a = [article_body[i] for i in label_a_index]

    # convert the article to JSON string
    chunk = json.dumps(article_a)

    responses = []

    difference = generate_difference(chunk, client, model=model)
    responses.append(
        {
            "Difference": difference,
        }
    )

    # Write the generated questions to the output JSON file
    with open(output_file, "w") as f:
        json.dump(responses, f, indent=4)

In [5]:
import pandas as pd

# read the label file
labels = pd.read_csv("labels_4o.csv", header=None, names=["label"])

In [6]:
# get the label index if labe is 1
label_a_index = list(labels[labels["label"] == 1].index)
# label_b_index = list(labels[labels["label"] == 9].index)
random.shuffle(label_a_index)

In [7]:
# label_a_index = label_a_index[:20]
label_a_index = [
    56,
    91,
    104,
    192,
    160,
    10,
    112,
    117,
    130,
    1,
    103,
    46,
    2,
    165,
    33,
    87,
    62,
    65,
    25,
    162,
]
label_a_index

[56,
 91,
 104,
 192,
 160,
 10,
 112,
 117,
 130,
 1,
 103,
 46,
 2,
 165,
 33,
 87,
 62,
 65,
 25,
 162]

In [8]:
input_file = "combined_Russo_Ukrainian_War.json"
output_file = "output_difference_7.json"

# Assuming you have your OpenAI API client initialized as `client`
process_batches(
    input_file,
    output_file,
    client,
    model="gpt-4o",
    label_a_index=label_a_index,
)

In [9]:
# print the value from JSON file

with open(output_file, "r") as f:
    output = json.load(f)[0]
    print(output["Difference"])

The news articles in English and Chinese present different narratives and emphases regarding the Ukraine-Russia conflict. Here are the key narrative differences:

1. **Tone and Perspective**:
   - **English Articles**: The tone is generally analytical and focused on the geopolitical implications of the conflict. They emphasize the strategic decisions of Western countries, the support provided to Ukraine, and the internal political dynamics within NATO and the EU.
   - **Chinese Articles**: The tone is more critical and often highlights the perceived failures and challenges faced by Ukraine and its Western allies. There is a focus on the internal divisions within NATO and the EU, and the articles often suggest that Western support for Ukraine is faltering or insufficient.

2. **Focus on Western Support**:
   - **English Articles**: These articles highlight the continued and substantial support from Western countries, including detailed descriptions of military aid packages and the strat

在分析这些英文和中文新闻文章时，可以发现它们在叙事上存在一些显著的差异。以下是一些主要的叙事差异：

1. **焦点和语调**：
   - **英文文章**：通常更注重报道事实和事件，语调较为中立。例如，关于美国对乌克兰的军事援助，英文文章详细列出了援助的具体内容和金额，并引用了官方声明。
   - **中文文章**：则更倾向于表达观点和情感，语调可能更为激烈。例如，中文文章中提到特朗普声称能在24小时内结束俄乌战争时，语气带有质疑和讽刺。

2. **信息的详细程度**：
   - **英文文章**：通常提供更详细的背景信息和数据。例如，关于北约对乌克兰的支持，英文文章详细列出了各国的立场和具体的援助措施。
   - **中文文章**：有时会省略一些具体细节，更注重传达整体观点和情绪。例如，中文文章中提到北约对乌克兰的支持时，更多的是强调支持的意义和影响，而不是具体的援助内容。

3. **叙事角度**：
   - **英文文章**：多从国际关系和政策角度进行分析，强调各国政府和国际组织的立场和行动。例如，关于北约峰会的报道，英文文章详细讨论了各国领导人的立场和会议的预期结果。
   - **中文文章**：则更多从民族情感和政治立场出发，强调国家利益和国际形势。例如，中文文章中提到芬兰总统对中国的依赖时，强调了中国在国际事务中的影响力和地位。

4. **引用和来源**：
   - **英文文章**：通常引用官方声明、专家意见和具体数据，信息来源较为多样和权威。例如，关于美国对乌克兰的援助，英文文章引用了国防部的声明和具体的援助数据。
   - **中文文章**：则更多引用国内媒体和政府官员的言论，信息来源相对集中。例如，关于特朗普的言论，中文文章引用了中国媒体的报道和评论。

5. **叙事结构**：
   - **英文文章**：结构较为严谨，通常按照事件发生的时间顺序或逻辑顺序进行报道，信息层次分明。例如，关于北约峰会的报道，英文文章按照会议的背景、各国立场和预期结果进行详细描述。
   - **中文文章**：结构相对灵活，可能会在报道中穿插评论和观点，信息呈现方式更为多样。例如，关于乌克兰战争的报道，中文文章可能会在描述战争进展的同时，加入对国际形势的评论和分析。

通过以上分析，可以看出英文和中文新闻文章在叙事上存在显著差异，这些差异反映了不同语言和文化背景下的新闻报道风格和信息传达方式。

## from Chinese prompt:

Sure! Here is the translation into English:

---

When analyzing these English and Chinese news articles, some significant narrative differences can be observed. Here are some main narrative differences:

1. **Focus and Tone**:
   - **English Articles**: Typically focus more on reporting facts and events with a neutral tone. For example, in articles about U.S. military aid to Ukraine, English articles detail the specific contents and amounts of the aid and cite official statements.
   - **Chinese Articles**: Tend to express opinions and emotions, and the tone may be more intense. For example, when Chinese articles mention Trump's claim that he could end the Russia-Ukraine war within 24 hours, the tone carries skepticism and sarcasm.

2. **Level of Detail**:
   - **English Articles**: Usually provide more detailed background information and data. For instance, regarding NATO's support for Ukraine, English articles list the positions of various countries and specific aid measures in detail.
   - **Chinese Articles**: Sometimes omit specific details, focusing more on conveying the overall viewpoint and sentiment. For example, when discussing NATO's support for Ukraine, Chinese articles emphasize the significance and impact of the support rather than the specific aid content.

3. **Narrative Perspective**:
   - **English Articles**: Often analyze from the perspective of international relations and policies, emphasizing the positions and actions of governments and international organizations. For example, reports on NATO summits in English articles discuss the stances of various leaders and the expected outcomes of the meetings in detail.
   - **Chinese Articles**: More often approach from the perspective of national sentiment and political stance, emphasizing national interests and the international situation. For instance, when Chinese articles mention the Finnish president's dependence on China, they highlight China's influence and status in international affairs.

4. **Citations and Sources**:
   - **English Articles**: Typically cite official statements, expert opinions, and specific data, with diverse and authoritative sources. For example, articles about U.S. aid to Ukraine cite statements from the Department of Defense and specific aid data.
   - **Chinese Articles**: More often cite statements from domestic media and government officials, with sources relatively concentrated. For example, articles about Trump's remarks cite reports and comments from Chinese media.

5. **Narrative Structure**:
   - **English Articles**: Have a more rigorous structure, usually reporting in chronological or logical order, with clear information hierarchy. For example, reports on NATO summits in English articles describe the background, various national stances, and expected outcomes of the meetings in detail.
   - **Chinese Articles**: Have a relatively flexible structure, possibly interspersing commentary and viewpoints in the reports, presenting information in a more diverse manner. For instance, reports on the Ukraine war in Chinese articles may include commentary and analysis on the international situation alongside descriptions of the war's progress.

Through the above analysis, it can be seen that there are significant narrative differences between English and Chinese news articles. These differences reflect the styles of news reporting and information dissemination in different language and cultural contexts.

## from English prompt:

The news articles in English and Chinese present different narratives and emphases regarding the Ukraine-Russia conflict. Here are the key narrative differences:

1. **Tone and Perspective**:
   - **English Articles**: The tone is generally analytical and focused on the geopolitical implications of the conflict. They emphasize the strategic decisions of Western countries, the support provided to Ukraine, and the internal political dynamics within NATO and the EU.
   - **Chinese Articles**: The tone is more critical and often highlights the perceived failures and challenges faced by Ukraine and its Western allies. There is a focus on the internal divisions within NATO and the EU, and the articles often suggest that Western support for Ukraine is faltering or insufficient.

2. **Focus on Western Support**:
   - **English Articles**: These articles highlight the continued and substantial support from Western countries, including detailed descriptions of military aid packages and the strategic importance of this support for Ukraine's defense.
   - **Chinese Articles**: These articles often question the effectiveness and sustainability of Western support. They emphasize delays in aid delivery, internal disagreements among Western allies, and the potential for reduced support in the future.

3. **Narrative on Ukraine's Military Situation**:
   - **English Articles**: The narrative is generally supportive of Ukraine's military efforts, highlighting successes and the resilience of Ukrainian forces. There is an emphasis on the strategic importance of continued Western support to ensure Ukraine's success.
   - **Chinese Articles**: The narrative often portrays Ukraine's military situation as dire, with significant challenges and setbacks. There is a focus on the difficulties faced by Ukrainian forces and the potential for a prolonged and unresolved conflict.

4. **Political Dynamics and Leadership**:
   - **English Articles**: These articles discuss the political dynamics within NATO and the EU, including the challenges of maintaining unity and the strategic decisions being made to support Ukraine. They also cover the political implications of the conflict for Western leaders.
   - **Chinese Articles**: These articles often highlight the political divisions and disagreements within NATO and the EU. They suggest that these divisions undermine the effectiveness of Western support for Ukraine and question the leadership and decision-making of Western countries.

5. **Role of Non-Western Actors**:
   - **English Articles**: The focus is primarily on the actions and decisions of Western countries and their impact on the conflict. There is less emphasis on the role of non-Western actors such as China or North Korea.
   - **Chinese Articles**: There is a significant emphasis on the role of non-Western actors, particularly China and North Korea. These articles often suggest that these countries could play a decisive role in the conflict and highlight their potential support for Russia.

6. **Future Outlook**:
   - **English Articles**: The outlook is cautiously optimistic, with an emphasis on the importance of continued Western support and the potential for Ukraine to achieve its strategic objectives.
   - **Chinese Articles**: The outlook is more pessimistic, suggesting that the conflict is likely to be prolonged and that Western support may wane over time. There is a focus on the potential for further escalation and the challenges faced by Ukraine and its allies.

In summary, the English articles tend to present a more supportive and strategic view of the conflict, emphasizing the importance of Western unity and continued support for Ukraine. In contrast, the Chinese articles are more critical, highlighting the challenges and divisions within the Western alliance and questioning the effectiveness and sustainability of Western support for Ukraine.